# Performance Comparison of Face Recognition Models (Assignment 3)

Nama:Najwan Yusnianda
NIM: 2408207010029

## 1. Prepare Dataset

### 1.1 Load Library

In [8]:
import os
import random
import shutil
import numpy as np
import pandas as pd
import requests
import cv2
import tensorflow as tf
from mtcnn import MTCNN
from imutils import paths
from tqdm import tqdm
from pathlib import Path
from deepface import DeepFace
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

### 1.2 Download Dataset

In [9]:
## import metadata
facescrub_df_actor = pd.read_csv('faceScrub/facescrub_actors.txt',delimiter='\t',header=None)
facescrub_df_actress = pd.read_csv('faceScrub/facescrub_actresses.txt',delimiter='\t',header=None)

#combine dataframe
facescrub_df=pd.concat([facescrub_df_actor,facescrub_df_actress],axis=0)

print(f"number of row (images) : {len(facescrub_df)}")

number of row (images) : 106865


In [10]:
# Group by the first column (name) and count the occurrences
name_count_df = facescrub_df.groupby(facescrub_df.columns[0]).size().reset_index(name='count')

# Rename the columns for clarity
name_count_df.columns = ['name', 'count']
name_count_df.head()
# Display the new dataframe
#name_count_df=name_count_df.sort_values(by='count', ascending=False)
print(f"number of unique faces (people): {len(name_count_df)}")
print(name_count_df.head())


number of unique faces (people): 531
            name  count
0  Aaron Eckhart    231
1     Adam Brody    200
2     Adam McKay    108
3   Adam Sandler    208
4  Adrianne León     62


In [11]:
def download_and_detect_faces(url, filename):
    detector = MTCNN()
    
    try:

        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        # Convert to numpy array 
        image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    
        #check if image
        if image is None:
            return 0

        # Detect faces in the image
        result = detector.detect_faces(image)
        if not len(result) ==1:
            #print(f"no face detected in {filename} number of face detected: {len(result)}")
            return 0
            
        # Only save the image if faces detected
        cv2.imwrite(filename, image)
        return 1
        
    except (requests.exceptions.RequestException, cv2.error) as e:
        #remove image if not contain face
        if os.path.exists(filename):
            try:
                os.remove(filename)
            except OSError:
                pass
        return 0
    except Exception as e:
        # Catch any other unexpected errors
        if os.path.exists(filename):
            try:
                os.remove(filename)
            except OSError:
                pass
        return 0
    

    

def get_image_sample(num_person,ntrain_person,ntest_person,nval_person,facescrub_df):
    current_num_person = 0

    list_person = []

    while current_num_person<num_person:
        current_ntrain_person = 0
        current_ntest_person = 0
        current_nval_person = 0
        ##generate angka random sebagai index dari dataset
        random_num = np.random.randint(0, facescrub_df.shape[0])
        ##ambil nama orang dari dataset
        
        current_person=facescrub_df.iloc[random_num,0]
        
        if current_person in list_person:
            continue
        else:
        ##buat dataset dengan data orang tersebut
            df_persons=facescrub_df[facescrub_df[0]==current_person]
            
            train_path = 'dataset/stagging/sampleset/'+current_person
            val_path = 'dataset/stagging/validationset/'+current_person
            test_path = 'dataset/stagging/testingset/'+current_person
            ##buat folder untuk orang tersebut
            if not os.path.exists(train_path):
                
                os.makedirs(train_path)
            if not os.path.exists(test_path):
                os.makedirs(test_path)

            len_train = print(f"dwonload image person={current_person} num_image={df_persons.shape[0]}")
            
            
            list_index_person=[]
            while current_ntrain_person<ntrain_person:
                    #get random index
                i = np.random.randint(0, df_persons.shape[0])
                if i in list_index_person:
                    continue
                else:
                    list_index_person.append(i)
                    ##ambil url gambar
                    url = df_persons.iloc[i,3]
                    ##download gambar
                    isdownload=download_and_detect_faces(url, train_path+'/'+current_person+'_'+str(current_ntrain_person+1)+'.jpg')
                    if isdownload==1:
                        current_ntrain_person+=1
                        if current_ntrain_person % 10 == 0:
                            print(f"sample_person set image added for {current_person}: {current_ntrain_person}")
            while current_nval_person<nval_person:
                i = np.random.randint(0, df_persons.shape[0])
                if i in list_index_person:
                    continue
                else:
                    list_index_person.append(i)    
                     ##ambil url gambar
                    url = df_persons.iloc[i,3]
                    ##download gambar
                    isdownload=download_and_detect_faces(url, val_path+'/'+current_person+'_'+str(current_nval_person+1)+'.jpg')
                    if isdownload==1:
                        current_nval_person+=1
                        if current_nval_person % 5 == 0:
                            print(f"val_person set image added for {current_person}: {current_nval_person}")
            while current_ntest_person<ntest_person:
                i = np.random.randint(0, df_persons.shape[0])
                if i in list_index_person:
                    continue
                else:
                    list_index_person.append(i)    
                     ##ambil url gambar
                    url = df_persons.iloc[i,3]
                    ##download gambar
                    isdownload=download_and_detect_faces(url, test_path+'/'+current_person+'_'+str(current_ntest_person+1)+'.jpg')
                    if isdownload==1:
                        current_ntest_person+=1
                        if current_ntest_person % 5 == 0:
                            print(f"test_person set image added fro {current_person}: {current_ntest_person}")
            list_person.append(current_person)
            current_num_person+=1
            print(f"=== [{current_num_person}/{num_person}] Person added: {current_person} ===")


In [ ]:
##run function to get image sample
get_image_sample(num_person=200,ntrain_person=20,ntest_person=5,nval_person=5,facescrub_df=facescrub_df)

dwonload image person=Robert Di Niro num_image=200
sample_person set image added for Robert Di Niro: 10
sample_person set image added for Robert Di Niro: 20
val_person set image added for Robert Di Niro: 5
test_person set image added fro Robert Di Niro: 5
=== [1/200] Person added: Robert Di Niro ===
dwonload image person=Wendie Malick num_image=219
sample_person set image added for Wendie Malick: 10
sample_person set image added for Wendie Malick: 20
val_person set image added for Wendie Malick: 5
test_person set image added fro Wendie Malick: 5
=== [2/200] Person added: Wendie Malick ===
dwonload image person=Jennie Garth num_image=245
sample_person set image added for Jennie Garth: 10
sample_person set image added for Jennie Garth: 20
val_person set image added for Jennie Garth: 5
test_person set image added fro Jennie Garth: 5
=== [3/200] Person added: Jennie Garth ===
dwonload image person=Linda Gray num_image=226
sample_person set image added for Linda Gray: 10
sample_person set i

### 1.2.2 Organize folder(optional if all faces allready download)

In [ ]:
import os
import shutil
import random
import cv2
from mtcnn import MTCNN

# Initialize MTCNN detector
detector = MTCNN()

# Define paths
base_folders = ["dataset/actors", "dataset/actress"]
train_folder = "dataset/stagging/sampleset"
validation_folder = "dataset/stagging/validationset"
test_folder = "dataset/stagging/testingset"

# Function to reset a folder
def reset_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
    os.makedirs(folder_path)

# Reset dataset folders
reset_folder(train_folder)
reset_folder(validation_folder)
reset_folder(test_folder)

# Collect all person folders
all_persons = []
for base_folder in base_folders:
    if os.path.exists(base_folder):
        persons = [
            os.path.join(base_folder, p) for p in os.listdir(base_folder)
            if os.path.isdir(os.path.join(base_folder, p))
        ]
        all_persons.extend(persons)
print(f"Found {len(all_persons)} persons.")

# Randomly select up to 200 persons
selected_persons = random.sample(all_persons, min(200, len(all_persons)))
print(f"Selected {len(selected_persons)} persons.")

def is_valid_face(image_path):
    """Check if an image contains exactly one detectable face."""
    try:
        image = cv2.imread(image_path)  # Read the image
        result = detector.detect_faces(image)  # Detect faces

        if len(result) == 1:  # Keep only images with exactly 1 face
            return True
        else:
            return False
    except Exception as e:
        return False

# Process each selected person
for person_path in selected_persons:
    person_name = os.path.basename(person_path)
    print(f"Processing {person_name}...")

    images = [img for img in os.listdir(person_path) if img.endswith((".jpeg", ".jpg", ".png"))]

    # Shuffle images
    random.shuffle(images)

    # Filter images with exactly one detected face
    valid_images = [img for img in images if is_valid_face(os.path.join(person_path, img))]

    # Skip if fewer than 20 valid images
    if len(valid_images) < 20:
        print(f"⚠️ Skipping {person_name}, not enough valid face images.")
        continue

    # Select max 25 valid images
    selected_images = valid_images[:30]
    train_images = selected_images[:20]  # 15 for training
    validation_images = selected_images[20:25]  # 5 for validation
    test_images = selected_images[25:]  # 5 for testing

    # Create person-specific directories
    train_person_path = os.path.join(train_folder, person_name)
    validation_person_path = os.path.join(validation_folder, person_name)
    test_person_path = os.path.join(test_folder, person_name)

    os.makedirs(train_person_path, exist_ok=True)
    os.makedirs(validation_person_path, exist_ok=True)
    os.makedirs(test_person_path, exist_ok=True)

    # Copy images to respective folders
    for img in train_images:
        shutil.copy(os.path.join(person_path, img), os.path.join(train_person_path, img))

    for img in validation_images:
        shutil.copy(os.path.join(person_path, img), os.path.join(validation_person_path, img))

    for img in test_images:
        shutil.copy(os.path.join(person_path, img), os.path.join(test_person_path, img))

    print(f"✅ Processed {person_name}: Train={len(train_images)}, Validation={len(validation_images)}, Test={len(test_images)}")

print("🎉 Dataset successfully filtered and organized!")


Selected 200 persons.
Processed Kassie_DePaiva: Train=20, Test=5
Processed Bradley_Cooper: Train=20, Test=5
Processed Cam_Gigandet: Train=20, Test=5
Processed Audra_McDonald: Train=20, Test=5
Processed Shannon_Kane: Train=20, Test=5
Processed Crystal_Chappell: Train=20, Test=5
Processed Jennette_McCurdy: Train=20, Test=5
Processed John_Malkovich: Train=20, Test=5
Processed Seth_Rogen: Train=20, Test=5
Processed Kris_Kristofferson: Train=20, Test=5
Processed John_Noble: Train=20, Test=5
Processed Catherine_Bell: Train=20, Test=5
Processed Sean_Bean: Train=20, Test=5
Processed Alyssa_Milano: Train=20, Test=5
Processed Jason_Behr: Train=20, Test=5
Processed Martin_Sheen: Train=20, Test=5
Processed Antonio_Banderas: Train=20, Test=5
Processed Tempestt_Bledsoe: Train=20, Test=5
Processed Ed_Harris: Train=20, Test=5
Processed Valerie_Harper: Train=20, Test=5
Processed Amaury_Nolasco: Train=20, Test=5
Processed Oliver_Platt: Train=20, Test=5
Processed Chyler_Leigh: Train=20, Test=5
Processed 

### 1.3 Process Image with MTCNN

In [3]:

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [15]:
def processed_mtcnn(detector,dirpath, dirdest):
    
    imagePaths = sorted(list(paths.list_images(dirpath)))
    for imagePath in tqdm(imagePaths):
        #path_split = imagePath.split(os.sep)
        path_split=Path(imagePath).parts
        name_person = path_split[-2]
        fn = path_split[-1].split('.')
        filename, fileformat = fn[0], fn[1]

        os.makedirs(dirdest, exist_ok=True)

        image = cv2.cvtColor(cv2.imread(imagePath), cv2.COLOR_BGR2RGB)
        result = detector.detect_faces(image)
        #print(f"Processing: {imagePath}, Faces detected: {len(result)}")

        for i in range(len(result)):
            bounding_box = result[i]['box']
            keypoints = result[i]['keypoints']

            bounding_box[0] = max(0, bounding_box[0])
            bounding_box[1] = max(0, bounding_box[1])

            person_dir = os.path.join(dirdest, name_person)
            os.makedirs(person_dir, exist_ok=True)
            path_save = os.path.join(person_dir, f"{filename}_{i}.{fileformat}")
            print(path_save)
            img = image[bounding_box[1]:bounding_box[1] + bounding_box[3],
                    bounding_box[0]:bounding_box[0] + bounding_box[2]]

            cv2.imwrite(path_save, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))


    print("Face detection and cropping completed!")

In [16]:
dirpaths=['dataset/stagging/sampleset','dataset/stagging/testingset']

for dirpath in dirpaths:
    processed_mtcnn(detector=MTCNN(),dirpath=dirpath,dirdest=dirpath.replace("stagging","mtcnn_faces"))

  2%|▏         | 2/100 [00:00<00:17,  5.46it/s]

dataset/mtcnn_faces/sampleset\Adrienne_Barbeau\Adrienne_Barbeau_9044_4231_0.jpeg
dataset/mtcnn_faces/sampleset\Adrienne_Barbeau\Adrienne_Barbeau_9045_4232_0.jpeg


  4%|▍         | 4/100 [00:00<00:14,  6.45it/s]

dataset/mtcnn_faces/sampleset\Adrienne_Barbeau\Adrienne_Barbeau_9059_4240_0.jpeg
dataset/mtcnn_faces/sampleset\Adrienne_Barbeau\Adrienne_Barbeau_9065_4243_0.jpeg


  6%|▌         | 6/100 [00:00<00:14,  6.69it/s]

...

dataset/mtcnn_faces/testingset\Alan_Alda\Alan_Alda_2225_1229_0.jpeg


 96%|█████████▌| 24/25 [00:07<00:00,  3.05it/s]

dataset/mtcnn_faces/testingset\Alan_Alda\Alan_Alda_2342_1315_0.jpeg


100%|██████████| 25/25 [00:07<00:00,  3.16it/s]

dataset/mtcnn_faces/testingset\Alan_Alda\Alan_Alda_2408_1362_0.jpeg
Face detection and cropping completed!


## 2.Face Verification with DeepFace 


Pengujian dilakukan pada google colab (https://colab.research.google.com/drive/1FJffmhncRhJePS87fkacBXv3gSxY8mvz?usp=sharing )

### 2.1 Extract Embedding


In [ ]:

def extract_embeddings(dataset_path, output_file, model_name="Facenet512"):
    """Extracts embeddings and saves them in a pickle file."""
    embeddings_dict = {}

    for person in os.listdir(dataset_path):
        person_dir = os.path.join(dataset_path, person)
        if not os.path.isdir(person_dir):
            continue  # Skip non-directory files

        embeddings_dict[person] = {}
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            try:
                embedding = DeepFace.represent(image_path, model_name=model_name, enforce_detection=False)[0]["embedding"]
                embeddings_dict[person][image_name] = np.array(embedding)
                print(f"✔️ Extracted embedding for: {image_name}")
            except Exception as e:
                print(f"❌ Error processing {image_path}: {e}")
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, "wb") as f:
        pickle.dump(embeddings_dict, f)
    print(f"✅ Embeddings saved to {output_file}")

    return embeddings_dict

### 2.2 Load Embedding

In [ ]:
def load_embeddings(file_path):
    """Loads the precomputed embeddings from a file."""
    with open(file_path, "rb") as f:
        embeddings = pickle.load(f)
    print(f"✅ Loaded embeddings from {file_path}")
    return embeddings

def verify_faces_with_embeddings(embeddings, person1, img1, person2, img2, threshold=0.4):
    """Compares precomputed embeddings for face verification."""
    if person1 not in embeddings or person2 not in embeddings:
        print(f"Error: Person not found in embeddings.")
        return False, None

    if img1 not in embeddings[person1] or img2 not in embeddings[person2]:
        print(f"Error: Image not found in embeddings.")
        return False, None

    emb1 = embeddings[person1][img1]
    emb2 = embeddings[person2][img2]

    distance = cosine(emb1, emb2)  # Cosine similarity
    verified = distance <= threshold

    return verified, distance

### 2.3 Run Verification

In [ ]:
def evaluate_performance(train_embeddings, test_embeddings, threshold, model="Facenet512",results_path='/content/drive/MyDrive/dataset_colab/output'):
    """Evaluates face verification using precomputed embeddings."""
    results = []
    results_dir = f"{results_path}/results_{threshold}"
    os.makedirs(results_dir, exist_ok=True)

    for test_person, test_images in test_embeddings.items():
        print(f"Processing test person: {test_person}")

        for train_person, train_images in train_embeddings.items():
            if test_person == train_person:
                for test_img, test_emb in test_images.items():
                    for train_img, train_emb in train_images.items():
                        distance = cosine(test_emb, train_emb)
                        verified = distance <= threshold

                        results.append({
                            'train': train_img,
                            'test': test_img,
                            'verified': verified,
                            'y_true': 1,  # Since it's the same person
                            'y_pred': 1 if verified else 0,
                            'distance': distance,
                            'model': model
                        })

        # Save per-person results
        df_person = pd.DataFrame(results)
        df_person.to_csv(os.path.join(results_dir, f"results_{test_person}_{threshold}_{model}.csv"), index=False)

    # Save overall results
    df_results = pd.DataFrame(results)
    df_results.to_csv(os.path.join(results_dir, f"results_{threshold}_{model}.csv"), index=False)

    return df_results

In [ ]:
train_dir=pathd+'/sampleset'
val_dir=pathd+'/validationset'
test_dir=pathd+'/testingset'
threshold=0.4
model



results_df=evaluate_performance(train_dir=train_dir,test_dir=test_dir,threshold=threshold)

## 3. Performance Evaluation

### 3.1 Calculate Metrics

In [1]:
def calculate_metrics(results_df, model_name, threshold,results_path='/content/drive/MyDrive/dataset_colab/output'):
    """Calculates and exports classification metrics."""
    y_true = results_df['y_true']
    y_pred = results_df['y_pred']

    cm = confusion_matrix(y_true, y_pred)
    TP = cm[1, 1]
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    metrics_df = pd.DataFrame({
        'Model': [model_name],
        'TP': [TP],
        'TN': [TN],
        'FP': [FP],
        'FN': [FN],
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1-Score': [f1]
    })

    eval_dir = f'{results_path}/results_{threshold}/eval'
    os.makedirs(eval_dir, exist_ok=True)
    metrics_df.to_csv(f'{eval_dir}/evalmetrics_{model_name}_{threshold}.csv', index=False)

    return metrics_df

In [2]:
def combine_results(dir_path):
    """Combines results from CSV files within a directory."""
    all_results = []
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)  # Form the complete file path
        if os.path.isfile(file_path) and file_name.endswith(".csv"):  # Check if it's a file and a CSV
            try:
                df = pd.read_csv(file_path)
                all_results.append(df)
            except FileNotFoundError:
                print(f"Warning: File not found - {file_path}")
            except pd.errors.EmptyDataError:
                print(f"Warning: Empty file - {file_path}")
    combined_df = pd.concat(all_results, ignore_index=True)
    return combined_df